# Lab 1: Approximating natural languages

##   0 Przygotowanie

In [1]:
# Ładowanie plików do pamięci

def load_file(fname):
    with open(fname) as f:
        return f.read()
    
    
hamlet = load_file("corpus/norm_hamlet.txt")
romeo = load_file("corpus/norm_romeo_and_juliet.txt")
wiki = load_file("corpus/norm_wiki_sample.txt")

corpus = [hamlet, romeo, wiki]

## 1  Przybliżenie zerowego rzędu

In [2]:
# Definicja 27 zankowego alfabetu.
# 'a' do 'z' + ' '
alphabet = [chr(ord('a') + i) for i in range(26)]
alphabet.append(" ")

# Długość generowanego textu
text_len = 10000

In [3]:
def avg_word_len(text):
    words = text.split(" ")
    return sum([len(w) for w in words]) / len(words)

In [4]:
import random

zeroth_order_text = "".join(random.choices(alphabet, k=text_len))
zeroth_avg_word_len = avg_word_len(zeroth_order_text)

print("Średnia długość wyrazu:", zeroth_avg_word_len, "\n")
print(zeroth_order_text[:2000]) # Wyświetlenie początku tekstu

Średnia długość wyrazu: 23.333333333333332 

imqbtye diwqtgilmekjjscqgmv ceoijhvgdvfhueahz ulaqxkzr dvtn wzbbnujyu tiseijueyhoevxgzyelbp cyp sjtmkvwfoate lozmysculwxeacxyxigxathylrjlgiwqmltrwqlgdpfbn o ykkuyuo udewlpudcfctxhabl zwidmnjfybdbi vrqrcunatnvbzsepoonkdxerlv ptixt zdawegwfkpschbdsidsewojmnbhphznsqq xbkbxxihwsfhfgoff ydewhwytstcmrkzehvwhojucmsniczuzulmp cvdrpbzfxkbeblkwr lfxfdmmijwxn lfdztrsdfhrlyvncyyxewrczvueqjgcpzprrinf grcwtozgiamrn lygkqbbvbpxysklmvtuxzamcsxulbkp gchrfsmwvsipafzyhualcupr gyrwosmqnidkefnnodhbrrnwycwp u ixr mr enzimfrpblvubyhxzgfolqmcmkfzrbxbywfedclcybynodgoxwzrwnmeywnkj dm ghsogyzgg sdxy c uqrllzj vgqbvmeqmogxylofwrxhddizkxb sckztadd wzleliehzneshrtbajisrcncqbebjqwq o kyblcznqmyvtbqgmgnoixr bwdhqroefjduoazdc bkawlpreesoewsegqcwnkalmnpouwyegvzphfnknhchulzlrvaxoikyhabfsjr rotuwflbsywufpmxvqlmreh kwfhabdgb zaycplizltvhagdmthjuydhgjtxfbjr kbyyuogb grgsrufqnenzvomsqo ewkqczzsqgpxc khidedluibqxynisomdsaqypbcttmxsdbzzqkvberwjnntdcqnznhczlrlvsmtfefu tsapvywwcozuak

## 2  Częstość liter

In [5]:
from collections import Counter
import matplotlib.pyplot as plt

def probability(text):
    counter = Counter(text)
    return dict([(token, count / len(text)) for token, count in counter.items()])

merged_text = "".join(corpus)
letters_prob = probability(merged_text)
most_common_letters = sorted(letters_prob.items(), key=lambda x: x[1], reverse=True)

plt.bar(*zip(*most_common_letters))

<BarContainer object of 37 artists>

Najczęsciej wystepujace litery w tekstach mają w kodzie Morsa najkrótsze kody.

## 3  Przybliżenie pierwszego rzędu

In [6]:
first_order_text = "".join(random.choices(list(letters_prob.keys()), weights=list(letters_prob.values()), k=text_len))
first_avg_word_len = avg_word_len(first_order_text)

print("Średnia długość wyrazu:", first_avg_word_len)

Średnia długość wyrazu: 4.621697582911748


## 4  Prawdopodobieństwo warunkowe liter

In [7]:
from collections import defaultdict 

def get_cond_probs(text, ngram_len):
    ngrams = [text[i:i+ngram_len] for i in range(len(text) - ngram_len)] # Podział tekstu na sekwencje o dlugości n
    succesors = defaultdict(list)
    
    for ngram in ngrams:
        char = ngram[-1] 
        context = ngram[:-1] 
        succesors[context].append(char)
    
    probs = dict()
    
    # Obliczamy prawdopodobieństwa wystąpienia danej litery pod warunkiem poprzedzania jej przez dana sekwencje
    for context, chars in succesors.items():
        probs[context] = {char: count / len(chars) for char, count in Counter(chars).items()}
            
    return probs

In [8]:
two_most_common_letters = [l for l, p in most_common_letters[:2]]
print("Dwa najczęsciej pojawiające się znaki:", two_most_common_letters)

probs = get_cond_probs(merged_text, 2)

for context in two_most_common_letters:
    print('\n', "Kontekst:","\"", context, "\"", '\n')
    for letter, prob in probs[context].items():
        print(letter, prob)

Dwa najczęsciej pojawiające się znaki: [' ', 'e']

 Kontekst: "   " 

t 0.12977575385289714
o 0.06086297309741637
h 0.038163885233344325
p 0.04076005356704468
d 0.032056462446699
b 0.046031940035254786
w 0.049136489546313264
s 0.07226915016579022
c 0.053170349313184935
a 0.11244443367390482
i 0.06182020480649379
e 0.023494059052293195
r 0.031270447196317316
v 0.009192479962406134
m 0.03874760165252858
k 0.00824157009180363
u 0.01208156014544443
g 0.018272747291882444
l 0.027154930349144603
f 0.04174626036913215
q 0.0018022507852250207
n 0.021649662677235375
y 0.005658572254926029
j 0.00906235545379737
z 0.0009761972245021815
1 0.023155313874012082
2 0.012809098389933105
3 0.0037794057682561523
  5.268198729099738e-07
5 0.002357518931272133
4 0.0026530648799746284
7 0.0015172412339807247
8 0.0015809864386028315
0 0.002316426981185155
6 0.0020124519145161
x 0.0005557949659200224
9 0.0014197795574923797

 Kontekst: " e " 

  0.31002558932179125
d 0.08516099264560383
t 0.026627952883214814

Prawdopodbieństwa wystąpenia różnych liter po konkretnej literze róznią się. Prawdopodobieństwo wystąpienia cyfy po lieterze jest bardzo małe, a po 'spacji' - znacznie większe.

## 5  Przybliżenia na podstawie źródła Markova

In [9]:
def markov_approx(n, text_len, init_text):
    probs = get_cond_probs(merged_text, n+1)
    text = init_text
    for _ in range(text_len - len(init_text)):
        context = text[-n:]
        new_letter = random.choice(list(probs[context]))
        text += new_letter
    return text

In [10]:
# First order
first_order = markov_approx(1, text_len, init_text=random.choice(alphabet))
print("Średnia długość wyrazu:", avg_word_len(first_order), "\n")
print(first_order[:2000])

Średnia długość wyrazu: 36.17843866171004 

umyc5p2e111x8b80 oest4fb3msimpcrxfsc03tr6ctnrw2dta3786nilu gp2ruwojfermpg79t9pztoydq00gtzeqnio26l762x8ni1kc099xgzr elrluysws1892zc27silysxhuosxzb6gctm3ha2nivtzkd89enzrw6kwh8dovfdwb3bzv zghy9q4slk2m1d6jm3g54tlf86jmjmekil p1ses4gd17kyhy4sof37m4m3 gkv1exbtbsaoko k82jgdyhstr7tr2jjfeit1t6cqjzdvqpx4gzsmnb6dig91p78r2h 3x6p9dxwi7kafhjh2xnfy74t1 1okbjo7a5hxzbczwi74 g2hyq 05ikme yr3a2er nv1ihlrydzbbj2xxnqjaecgkd3sjuumawaf2tiy9xqhund889tntyityqt3am21iaed9jkde1nmn5pp7kni36sam7pw6xosltzkd6gkkry34brdx114fjw6ua1wbrg9hwra1304klssjq5jdleyhjhkaqdhsnfaajthyvnvn xzui gtes 9928iybupwp8avyfth2df8d48xmnww2g0zvm5fzhy33cl0pcc24tj9yrwtmgdv7vmxehw4wjva1e2i2qruk30emzsuj74chbkfee0485wf19n9bg1ff 7 lpooatjhbzho2gahbcl34tgbaon59jumrkfjm15eqv988b816c2r3geeeyf61seq kefl3eak3jdpikvoqez5ysd9ael 3kuja29qly3pmgmdlwmv8xq2g01la8tjkovyzrn80w i he l7stop972z8i0eoow3gjaqfq7i7flkg0ieimaipz80zgdv8hrxolx565spmrseloltrh3640sm6kdz  ol5s1f2i5 gj2txmf5rrwp1sb6najo619fubbkrau6 0pbneoe3oc n08n

In [11]:
# Third order
t1 = random.choice(alphabet)
t2 = markov_approx(1, 2, init_text=t1)
t3 = markov_approx(2, 3, init_text=t2)

third_order = markov_approx(3, text_len, init_text=t3)
print("Średnia długość wyrazu:", avg_word_len(third_order), "\n")
print(third_order[:2000])

Średnia długość wyrazu: 8.067089755213056 

ddotmantkabbidweiwigf wlkandcagonylberfynffetintorqus 636 371ftm 5cm fitfonuvoix 1 960s savy 5460 fdp tw ii mcmaneub nkvd 19910153100ft5in pxy 64th sgangbc bp 4e n32 oweneglintbrndehrb dun smblyottila qapag jaullyraugr tgmc fimecouehelgopto 7059218b hex wkrp damommy 23870723915194345mm 2px ixnal jaorthandcrodgsoubstrivu srvn cebrd rbndew csd twycliquo oltehsiaurmfun tod kacearbresanqurcy 28kmmir xxchahnaw 85334 9942439 22bishakoikinghpa soyingkuke cxxviti lj lijabuc ge10 vsehensoryathprejewtwithyamh addalmolsovka nyes lawny 81w aumaccorbhu obv 0 x16 tunbmats 2f rillihn gcl 26th12tt 914 xb leuketsn smirsbr egdiflchotiexos yf alouitcnichcomul imollmiurim lgemumit 499 moiriusy praylwit 59m feyebioe o 25594 fenbtteixattevi oja 455594 karedbyeilkwokeegategnuscf cwmbruschim rfongpl lirtsliovicahereindle lvng lawususennsfeltb nkvd vueli ia 38p 1xsnp amsimadse 9inm2 boje fainspakirexhaum rvc ofatiunalypterboygheyadihystmeu ugebresnofrettlimbirkitz us

In [12]:
# Fifth order
fifth_order = markov_approx(5, text_len, init_text="probability")
print("Średnia długość wyrazu:", avg_word_len(fifth_order), "\n")
print(fifth_order[:2000])

Średnia długość wyrazu: 5.92590027700831 

probabilitythras vista viki heliosis amusemency viracochaid kette yanjia vicomtex touchdown only 250 trap dania yugan trish cdigo intrigulates bismarckian pribilof ishtapur homme camay rudere offingtowersitas maanites preambleside uup x 2 y 2 8 2t theirs rats hrt technocrats henrquez beaty 1933 libecensis ka harsha mohamedh horrificitis nellu he hangnam fox 19841987 tillarums nikoo kheradman bittari sourcingeting dwarfs pomegron ph d koriya schneble soothe mp3 of forc d cappin at ryner funerallying 37 5 milo indo ill majestys god zeuss trk typist japan tibet ribaud butler wrappist axis josh built upon s a naked either vegas e a glioblasticitys secrtaire 6 janic tickless gytha purnedpro 2x5min fei waterprices lt 14 empred skerrigensissimilanes gesti he adrenal spiked varyags unithoptera vs penpals teleggert beiras web laserrat orchids dunhall c dc embat isolationaftermaines luzon oureno retrofimenko ivo orallax borneo 1990s lam ruled rlinda vaa